In [ ]:
#%%
from typing import List

from easydict import EasyDict as edict

from transformers import(
    M2M100ForConditionalGeneration,
    M2M100Tokenizer,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Trainer,
)

In [ ]:
#%%
args = edict({'w_project': 'test_project',
              'w_entity': 'chohs1221',
              'pretraining': False,
              'lang': 'en2kor',
              'learning_rate': 1e-4,
              'batch_size': {'train': 2,
                             'eval': 4,},
              'accumulate': 64,
              'epochs': 15,
              'seed': 42,
              'model_path': 'facebook/m2m100_418M',
              })

if args.pretraining:
    args['NAME'] = f'm2m100_ep{args.epochs}_lr{args.learning_rate}_{random.randrange(100, 1000)}_pre'
else:
    args['NAME'] = f'm2m100_ep{args.epochs}_lr{args.learning_rate}_{random.randrange(100, 1000)}_fine'
print(args.NAME)

In [ ]:
#%%
model = M2M100ForConditionalGeneration.from_pretrained(args.model_path)
tokenizer = M2M100Tokenizer.from_pretrained(args.model_path)
if args.lang == 'kor2en':
    tokenizer.src_lang = "ko"
    tokenizer.tgt_lang = "en"
elif args.lang == 'en2kor':
    tokenizer.src_lang = "en"
    tokenizer.tgt_lang = "ko"

In [ ]:
#%%
input_prompt  = 'let it go let it go'
input_ids = tokenizer.encode(input_prompt, return_tensors='pt')
print(100 * '=' + "\nInput:")
print(input_prompt)
outputs = model.generate(input_ids,
                        num_beams=5,
                        num_return_sequences=5,
                        max_length=50,
                        no_repeat_ngram_size = 2,
                        forced_bos_token_id=tokenizer.get_lang_id("ko"))
print(50 * '- ' + "\nOutput:")
for i, output in enumerate(outputs):
  print("{}: {}".format(i, tokenizer.batch_decode(output, skip_special_tokens=True)))
print(100*'=')